# Mocking

Sometimes our programmes depend on external circumstances, but during unit testing we shouldn't worry about whether they work. And only test our code. So there are ways to change the behaviour of functions called in the programs being tested, to make it convenient for us to do the testing.

## Use case

It's a bit of an advanced concept, so for better understanding the content of this page starts with the user case. This will help you understand why this slightly tricky option can be so useful. The following sections provide more detailed experiments with specific functions of mocking in tests.

Suppose you have service that answers some information about user. Just like the function in the next cell:

In [1]:
import requests

def request_user(user_id):
    response = requests.get(f"https://im_not_exist/{user_id}")
    if response.ok:
        return response.text
    else:
        return "Fail!"

Now we have to test this code. But you don't need to rely on any particular service in your tests, and you know that the requesting function under consideration uses `requests.get` to access information about the user from the service. So the best option for you here is to mock the `requests.get` function. So for test you change it behaviour in benificial for you way.

To do this, use the following syntax `unittest.mock.patch("<module under tests>.<name of the function to mock>")` as an object for the context manager. You must specify a module to change the behaviour of the function for that module only.

So such an option is implemented in the following cell. We've replaced the `request.get` function in the `request_user` module for test purposes. And check with which arguments it was called with and specify the properties of its output for particular case.

In [2]:
import unittest
from unittest.mock import patch

class TestUserReques(unittest.TestCase):
    
    def test_success_case(self):
        '''
        Sucessfull case. Suppose we got nice
        repsonse from requests.get in request_user function.
        '''
        with patch("__main__.requests.get") as mocked_get:
            # set that output of the request_user.requests.get
            # will have properties `ok == True` `text=="Success`
            mocked_get.return_value.ok = True
            mocked_get.return_value.text = "Success"
            
            response = request_user("Fedor")
            # request_user should have called `get` with specific url
            mocked_get.assert_called_with("https://im_not_exist/Fedor")
            # and "text" property have to be just like we specified
            self.assertEqual(response, "Success")

    def test_fail_case(self):
        '''
        Fail case. Suppose we got bad response
        from requests.get in the request_user function.
        '''
        with patch("__main__.requests.get") as mocked_get:
            # set that output of the request_user.requests.get
            # will have properties `ok == False` `text=="Success`
            # actually we don't really care wich exactly value
            # does text property have but unsver from always
            # have to be "Fail!"
            mocked_get.return_value.ok = False
            mocked_get.return_value.text = "Success"
            
            response = request_user("Ekaterina")
            # request_user should have called `get` with specific url
            mocked_get.assert_called_with("https://im_not_exist/Ekaterina")
            # but answer anyway have to be "Fail!" 
            self.assertEqual(response, "Fail!")

ans = unittest.main(argv=[''], verbosity=2, exit=False)
del TestUserReques

test_fail_case (__main__.TestUserReques)
Fail case. Suppose we got bad response ... ok
test_success_case (__main__.TestUserReques)
Sucessfull case. Suppose we got nice ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


## Called with

If you need to check which arguments were passed to the mocked function, you can use the `assert_called_with(<supposed arguments>)` method of the patch object.

Lets test it on really basic python function `sum`. In the following cell we have just defined wrapper for it.

In [3]:
def sum_wrapper(numbers):
    return sum(numbers)

Tests will mock `sum` funtion. In both tests to `sum_wrapper` was passed `[1,2,3]` list. But in second case we use `[1,2,5]` in `assert_called_with`.

In [4]:
import unittest
from unittest.mock import patch

class TestCalledWith(unittest.TestCase):
    def test_ok(self):
        with patch("__main__.sum_wrapper") as mocked_sum:
            sum_wrapper([1,2,3])
            mocked_sum.assert_called_with([1,2,3])

    def test_fail(self):
        with patch("__main__.sum_wrapper") as mocked_sum:
            sum_wrapper([1,2,3])
            mocked_sum.assert_called_with([1,2,5])      

ans = unittest.main(argv=[''], verbosity=2, exit=False)
del TestCalledWith

test_fail (__main__.TestCalledWith) ... FAIL
test_ok (__main__.TestCalledWith) ... ok

FAIL: test_fail (__main__.TestCalledWith)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_19381/616931396.py", line 13, in test_fail
    mocked_sum.assert_called_with([1,2,5])
  File "/usr/lib/python3.10/unittest/mock.py", line 929, in assert_called_with
    raise AssertionError(_error_message()) from cause
AssertionError: expected call not found.
Expected: sum_wrapper([1, 2, 5])
Actual: sum_wrapper([1, 2, 3])

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=1)


So one test passed successfully because the supposed argument in `assert_called_with` matches the argument passed to `sum_wrapper`. But in the second case we got "Fail" because it doesn't.